In [2]:
import glob
import polars as pl
import pandas as pd
from tqdm import tqdm
from datetime import timedelta

In [3]:
tracker_files = glob.glob('data/final_apparel_tracker_data_08_action_widget/*/*.parquet')
order_files = glob.glob('data/final_apparel_orders_data_07/*/*.parquet')
items_files = glob.glob('data/ml_ozon_recsys_train_final_apparel_items_data/*.parquet')
test_user_ids = pd.read_parquet('data/ml_ozon_recsys_test.snappy.parquet')
test_user_ids_set = set(test_user_ids['user_id'].to_list())

## Orders

In [4]:
all_dfs = []
for i in tqdm(range(len(order_files)), desc='Processing files'):
    file_path = order_files[i]

    df = pl.read_parquet(
        file_path, 
        columns=['user_id', 'item_id', 'created_timestamp', 'last_status', 'last_status_timestamp']
    ).filter(pl.col('created_timestamp') > pl.datetime(2025, 5, 21)) # from may
    all_dfs.append(df)

orders = pl.concat(all_dfs)

Processing files: 100%|██████████| 33/33 [00:02<00:00, 16.29it/s]


In [5]:
orders.filter(pl.col('user_id') == 1)

user_id,item_id,created_timestamp,last_status,last_status_timestamp
i32,i32,datetime[ns],str,datetime[ns]
1,129629015,2025-07-12 10:20:39.330,"""proccesed_orders""",2025-07-12 12:51:41
1,59149470,2025-07-12 10:20:23.433,"""proccesed_orders""",2025-07-12 13:28:09
1,57175120,2025-07-13 06:38:39.506,"""proccesed_orders""",2025-07-13 08:39:12
1,86451006,2025-06-02 17:46:48.806,"""delivered_orders""",2025-06-03 15:46:09
1,86451006,2025-06-02 17:48:38.060,"""canceled_orders""",2025-06-15 11:55:42
…,…,…,…,…
1,159338187,2025-06-29 05:28:08.546,"""canceled_orders""",2025-07-01 11:15:32.830
1,110740487,2025-06-06 13:14:38.956,"""delivered_orders""",2025-06-12 19:16:24
1,272664684,2025-06-29 05:57:23.203,"""delivered_orders""",2025-07-01 10:15:19


In [5]:
orders['created_timestamp'].max()

datetime.datetime(2025, 7, 15, 23, 59, 59, 50000)

In [42]:
item_ids_from_may = orders['item_id'].to_list()

In [ ]:
# Общая информация о данных
print("Размер данных:", orders.shape)
print("\nИнформация о столбцах:")
print(orders.schema)
print("\nОписательная статистика:")
print(orders.describe())

# Анализ временных меток
print("\nАнализ временных меток:")
for col in ['created_timestamp', 'last_status_timestamp']:
    print(f"\n{col}:")
    print("Минимальная дата:", orders[col].min())
    print("Максимальная дата:", orders[col].max())
    print("Количество пропусков:", orders[col].null_count())

# Анализ статусов
print("\nРаспределение статусов:")
print(orders['last_status'].value_counts())

Размер данных: (20362338, 5)

Информация о столбцах:
Schema({'user_id': Int32, 'item_id': Int32, 'created_timestamp': Datetime(time_unit='ns', time_zone=None), 'last_status': String, 'last_status_timestamp': Datetime(time_unit='ns', time_zone=None)})

Описательная статистика:
shape: (9, 6)
┌────────────┬─────────────┬──────────────┬──────────────────┬──────────────────┬──────────────────┐
│ statistic  ┆ user_id     ┆ item_id      ┆ created_timestam ┆ last_status      ┆ last_status_time │
│ ---        ┆ ---         ┆ ---          ┆ p                ┆ ---              ┆ stamp            │
│ str        ┆ f64         ┆ f64          ┆ ---              ┆ str              ┆ ---              │
│            ┆             ┆              ┆ str              ┆                  ┆ str              │
╞════════════╪═════════════╪══════════════╪══════════════════╪══════════════════╪══════════════════╡
│ count      ┆ 2.0362338e7 ┆ 2.0362338e7  ┆ 20362338         ┆ 20362338         ┆ 20362338         │
│ 

In [ ]:
status_changes = (
    orders
    .group_by(['user_id', 'item_id'])
    .agg([
        pl.col('last_status').unique().alias('unique_statuses')
    ])
    .filter(
        pl.col('unique_statuses').list.contains('canceled_orders') &
        (pl.col('unique_statuses').list.contains('delivered_orders') | 
         pl.col('unique_statuses').list.contains('proccesed_orders'))
    )
)

print("Количество товаров с изменением статуса:", status_changes.height)

Количество товаров с изменением статуса: 350086


In [ ]:
# Простой поиск пар user_id + item_id с обоими статусами
simple_examples = (
    orders
    .filter(pl.col('last_status').is_in(['delivered_orders', 'proccesed_orders', 'canceled_orders']))
    .group_by(['user_id', 'item_id'])
    .agg([
        pl.col('last_status').unique().alias('unique_statuses')
    ])
    .filter(
        pl.col('unique_statuses').list.contains('canceled_orders') &
        (pl.col('unique_statuses').list.contains('delivered_orders') | 
         pl.col('unique_statuses').list.contains('proccesed_orders'))
    )
)

print("Найдено пар user-item с изменением статуса:", simple_examples.height)

# Показываем конкретные пары
if simple_examples.height > 0:
    print("\nПервые 10 пар user_id + item_id:")
    for i, row in enumerate(simple_examples.head(10).iter_rows(named=True)):
        print(f"{i+1}. User: {row['user_id']}, Item: {row['item_id']}, Статусы: {row['unique_statuses']}")

Найдено пар user-item с изменением статуса: 350086

Первые 10 пар user_id + item_id:
1. User: 634421, Item: 153150280, Статусы: ['canceled_orders', 'delivered_orders']
2. User: 3758341, Item: 307165772, Статусы: ['canceled_orders', 'delivered_orders']
3. User: 2721471, Item: 48130003, Статусы: ['canceled_orders', 'proccesed_orders']
4. User: 4617841, Item: 197659645, Статусы: ['canceled_orders', 'delivered_orders']
5. User: 6070, Item: 275527247, Статусы: ['delivered_orders', 'canceled_orders']
6. User: 1969150, Item: 282101440, Статусы: ['delivered_orders', 'canceled_orders']
7. User: 155641, Item: 198128712, Статусы: ['proccesed_orders', 'canceled_orders']
8. User: 3614100, Item: 134537210, Статусы: ['delivered_orders', 'canceled_orders']
9. User: 1748731, Item: 240052276, Статусы: ['delivered_orders', 'canceled_orders']
10. User: 2969051, Item: 166045999, Статусы: ['delivered_orders', 'canceled_orders']


In [5]:
canceled_then_delivered = (
    orders
    .sort(['user_id', 'item_id', 'created_timestamp'])
    .group_by(['user_id', 'item_id'])
    .agg([
        pl.col('last_status').alias('status_sequence'),
        pl.col('created_timestamp').alias('timestamps'),
        pl.col('last_status').first().alias('first_status'),
        pl.col('last_status').last().alias('last_status')
    ])
    .filter(
        (pl.col('first_status') == 'delivered_orders') &
        (pl.col('last_status') == 'canceled_orders')
    )
)

print("Количество случаев с последовательностью ['canceled_orders', 'delivered_orders']:", canceled_then_delivered.height)

Количество случаев с последовательностью ['canceled_orders', 'delivered_orders']: 34618


In [ ]:
orders.filter(pl.col('user_id') == 634421)

user_id,item_id,created_timestamp,last_status,last_status_timestamp
i32,i32,datetime[ns],str,datetime[ns]
634421,289958675,2025-01-05 13:46:36.086,"""delivered_orders""",2025-02-09 16:14:02.617
634421,313204882,2025-01-09 10:34:24.950,"""delivered_orders""",2025-01-30 15:25:48.210
634421,289803477,2025-01-09 10:34:24.950,"""delivered_orders""",2025-02-02 11:15:22.623
634421,259460427,2025-01-05 13:37:55.413,"""canceled_orders""",2025-01-05 13:39:27.130
634421,10594724,2025-01-04 15:13:44.930,"""delivered_orders""",2025-01-20 17:42:14.513
…,…,…,…,…
634421,133696227,2025-01-05 13:37:55.413,"""canceled_orders""",2025-01-05 13:39:27.130
634421,153150280,2025-01-05 13:45:09.506,"""delivered_orders""",2025-01-23 15:43:15.060
634421,153150280,2025-01-05 13:37:55.413,"""canceled_orders""",2025-01-05 13:39:27.130


In [ ]:
orders.filter((pl.col('user_id') == 1748731) & (pl.col('item_id') == 240052276))

user_id,item_id,created_timestamp,last_status,last_status_timestamp
i32,i32,datetime[ns],str,datetime[ns]
1748731,240052276,2025-06-10 07:34:37.030,"""canceled_orders""",2025-06-14 05:43:38
1748731,240052276,2025-06-10 01:56:20.003,"""delivered_orders""",2025-06-12 19:24:22


In [ ]:
orders.filter((pl.col('user_id') == 6070) & (pl.col('item_id') == 275527247))

user_id,item_id,created_timestamp,last_status,last_status_timestamp
i32,i32,datetime[ns],str,datetime[ns]
6070,275527247,2025-01-25 17:37:24.863,"""canceled_orders""",2025-01-25 17:40:55.377
6070,275527247,2025-01-25 17:43:57.740,"""delivered_orders""",2025-02-01 12:10:44


In [6]:
max_date = orders['created_timestamp'].max()

# Вычисляем границу двухнедельного периода
two_weeks_ago = max_date - timedelta(days=14)

# Заказы за последние две недели
recent_orders = orders.filter(pl.col('created_timestamp') >= two_weeks_ago)

# Расчет показателей
total_orders = orders.height
total_recent_orders = recent_orders.height
unique_recent_users = recent_orders['user_id'].n_unique()

print(f"Доля заказов за последние две недели: {total_recent_orders/total_orders:.2%}")
print(f"Уникальных пользователей за последние две недели: {unique_recent_users}")

Доля заказов за последние две недели: 7.24%
Уникальных пользователей за последние две недели: 346759


In [9]:
max_date = orders['created_timestamp'].max()

# Вычисляем границу двухнедельного периода
delta = max_date - timedelta(days=55)

# Заказы за последние две недели
recent_orders = orders.filter(pl.col('created_timestamp') >= delta)

# Расчет показателей
total_orders = orders.height
total_recent_orders = recent_orders.height
unique_recent_users = recent_orders['user_id'].n_unique()

print(f"Доля заказов за последние две недели: {total_recent_orders/total_orders:.2%}")
print(f"Уникальных пользователей за последние две недели: {unique_recent_users}")

Доля заказов за последние две недели: 32.55%
Уникальных пользователей за последние две недели: 635813


Учитывая сезонность и климат, логично учитывать тренды покупки товаров с 22 мая.

## Items

In [11]:
items = pl.read_parquet(
    items_files,
    columns=["item_id", "itemname"]  # Указываем только нужные столбцы
)

In [31]:
items_1 = pl.read_parquet(items_files[0])

In [43]:
items_1

item_id,itemname,attributes,fclip_embed,catalogid,variant_id,model_id
i32,str,list[struct[4]],list[f32],i64,i32,i32
400301,"""Очки солнцезащитные""","[{""Type"",""Очки солнцезащитные"",false,true}, {""Brand"",""Нет бренда"",false,true}, … {""ColorBase_Forfilter"",""Розовый"",false,false}]","[0.320239, 0.117663, … 0.038868]",17080,243954236,33306429
678602,"""Футболка GOLDUSTIM Футболка же…","[{""Type"",""Футболка"",false,true}, {""Brand"",""GOLDUSTIM"",false,true}, … {""ColorBase_Forfilter"",""Розовый"",false,false}]","[0.436009, 0.633967, … -0.176413]",7508,8039633,27057398
746401,"""Худи Patagonia""","[{""Type"",""Худи"",false,true}, {""Brand"",""Patagonia"",false,true}, … {""ColorBase_Forfilter"",""Серый"",false,false}]","[0.474528, -0.47278, … 0.553977]",7555,120078071,11645051
872829,"""Комплект носков adidas 3S C Sp…","[{""Type"",""Комплект носков"",false,true}, {""Brand"",""adidas"",false,true}, … {""ColorBase_Forfilter"",""Белый"",false,false}]","[-0.069872, -2.280283, … 0.19949]",36563,247120069,38088455
893255,"""Жилет Buy & Style жилеты""","[{""Type"",""Жилет"",false,true}, {""Brand"",""Buy & Style"",false,true}, … {""ColorBase_Forfilter"",""Черный"",false,false}]","[0.124888, 0.546629, … 0.108914]",7535,327822613,19339929
…,…,…,…,…,…,…
338030820,"""Комплект одежды EMBERENS""","[{""Type"",""Комплект одежды"",false,true}, {""Brand"",""EMBERENS"",false,true}, … {""ColorBase_Forfilter"",""Серый"",false,false}]","[0.662005, -0.307679, … -0.079549]",7536,252544130,16863152
338096201,"""Футболка Разное""","[{""Type"",""Футболка"",false,true}, {""Brand"",""Нет бренда"",false,true}, … {""ColorBase_Forfilter"",""Черный"",false,false}]","[-0.664809, -0.895367, … -0.513037]",7508,290553419,31414259
338153876,"""Футболка Frutto Rosso однотонн…","[{""Type"",""Футболка"",false,true}, {""Brand"",""Frutto Rosso"",false,true}, … {""ColorBase_Forfilter"",""Бежевый"",false,false}]","[0.202864, 0.494631, … -0.388542]",7508,148237041,37736429


In [37]:
items_1.filter(pl.col('itemname') == 'Футболка')[0,2]

""
struct[4]
"{""Type"",""Футболка"",false,true}"
"{""Brand"",""Нет бренда"",false,true}"
"{""Annotation"",""Футболка унисекс (мужская и женская) белая фильмы Лицехват (Чужой, Alien, Ностальгия, 90-ые, horror, Гигер) - 38219 Футболка - это неотъемлемая часть гардероба: универсальная, практичная, стильная, сочетается с любой вещью.<br> Классический силуэт одновременно удобен в движении, так как достаточно свободен, но в то же время нигде ничего не топорщится и футболка смотрится аккуратно.<br> Трикотаж футболки изготовлен из полиэфирной пряжи по структуре и свойствам максимально похожей на хлопок, что позволяет коже дышать и делает ткань приятной к телу. При этом материал очень быстро сохнет и не мнется - что делает футболку универсальной для повседневной носки и спорта.<br> Печать принта осуществляется методом сублимации (прямой перенос краски на ткань), что делает принт долговечным.<br> <br> Уход:<br> Стирать при температуре не выше 40 градусов.<br> Гладить при температуре не выше 110 градусов.<br> Не использовать отбеливатель<br> <br> Детали:<br> Футболка имеет круглый вырез горловины, длина до линии бедер<br> <br> Тип нанесения:<br> Сублимация на ткани<br>"",false,true}"
"{""ColorBase"",""белый"",true,true}"
"{""ColorName"",""Футболка унисекс (мужская и женская) белая фильмы Лицехват (Чужой, Alien, Ностальгия, 90-ые, horror, Гигер) - 38219"",true,true}"
…
"{""Composition"",""50% Хлопок;50% Полиэстер"",false,true}"
"{""Season"",""На любой сезон"",false,true}"
"{""FilterSizeINT"",""XXL"",false,false}"


In [38]:
items_1.filter(pl.col('itemname') == 'Футболка')[1,2]

""
struct[4]
"{""Type"",""Футболка"",false,true}"
"{""Brand"",""Нет бренда"",false,true}"
"{""Annotation"",""<p>Футболка ""если ваше счастье не в деньгах шлите их мне"" - это отличный способ выразить свою индивидуальность и чувство юмора. Она станет отличным дополнением к любому образу, будь то повседневный или спортивный стиль.</p> <p>Эта футболка выполнена из высококачественного хлопка, который обеспечивает комфорт и долговечность. Материал приятен на ощупь, хорошо пропускает воздух и не вызывает раздражения кожи. Благодаря этому футболка идеально подходит для повседневной носки.</p> <p>Особенностью этой футболки является её дизайн. На передней части футболки расположена вышивка в виде забавной надписи. Надпись выполнена яркими нитками, которые не выцветают со временем. Это делает футболку не только стильной, но и практичной. Футболка доступна в различных размерах, что позволяет подобрать идеальный вариант для любого человека.</p> <p>Если вы ищете оригинальный подарок для друга или близкого человека, то эта футболка станет отличным выбором. Она будет радовать своего владельца каждый день, напоминая о вас и вашем чувстве юмора.</p> <p>Материал футболки - 100% хлопок плотностью 190 грамм на метр квадратный. Такая плотность ткани обеспечивает изделию прочность и износостойкость, а также позволяет коже дышать. Футболка сохраняет форму после стирки и не теряет цвет.</p> <p>Футболка унисекс, подойдёт как мужчинам, так и женщинам. Её можно сочетать с различными стилями одежды и носить в любое время года.</p>"",false,true}"
"{""ColorBase"",""черный"",true,true}"
"{""ColorName"",""черный"",true,true}"
…
"{""SleeveLength"",""Короткий"",false,true}"
"{""FilterSizeINT"",""XL"",false,false}"
"{""Name"",""Футболка ""если ваше счастье не в деньгах шлите их мне"""",false,false}"


In [35]:
items_1.select(pl.col('itemname').value_counts(sort=True))

itemname
struct[2]
"{""Футболка"",2599}"
"{""Кроссовки"",522}"
"{""Платье"",439}"
"{""Комплект одежды"",369}"
"{""Рубашка"",291}"
…
"{""Кардиган SEVINGIN"",1}"
"{""Блузка People in Popsi блузки"",1}"
"{""Рубашка cookie Школа"",1}"


### Оцениваем результаты из cooccurrence_neighbors_df

In [29]:
ids = neighbors_of_viewed[17,1].to_list()

In [30]:
items.filter(pl.col('item_id').is_in(ids)).select('itemname').to_series().to_list()

['Топ ТЕЛОДВИЖЕНИЯ Топ женский',
 'Майка Befree',
 'Топ ТЕЛОДВИЖЕНИЯ Топ женский',
 'Топ ТЕЛОДВИЖЕНИЯ Топ женский',
 'Майка Befree',
 'Майка Magic Lady CL',
 'Майка Befree',
 'Майка Magic Lady',
 'Майка Befree',
 'Майка Magic Lady',
 'Майка Befree',
 'Топ ТЕЛОДВИЖЕНИЯ Топ женский',
 'Майка Befree',
 'Майка Magic Lady',
 'Майка Befree',
 'Майка Befree',
 'Майка Befree',
 'Майка Befree',
 'Майка Befree',
 'Топ Befree',
 "Майка RIDAN'S топ бельевая на тонких бретельках",
 'Топ ТЕЛОДВИЖЕНИЯ Топ женский',
 'Топ ТЕЛОДВИЖЕНИЯ Топ женский',
 'Топ ТЕЛОДВИЖЕНИЯ Топ женский',
 'Майка Magic Lady',
 'Майка Befree',
 'Майка Magic Lady',
 'Топ Befree',
 'Майка Magic Lady',
 'Майка Befree',
 'Майка Befree',
 'Топ ТЕЛОДВИЖЕНИЯ Топ женский',
 'Топ ТЕЛОДВИЖЕНИЯ Топ женский',
 'Майка Befree',
 'Майка Befree',
 'Топ ТЕЛОДВИЖЕНИЯ Топ женский',
 'Майка Befree',
 'Топ ТЕЛОДВИЖЕНИЯ Топ женский',
 'Майка Befree',
 'Майка Befree',
 'Майка Magic Lady',
 'Майка Befree',
 'Топ Befree',
 'Майка Befree',
 'Майка Befr

In [2]:
neighbors_of_viewed = pl.read_parquet('candidate_cache/neighbors_of_viewed_items.parquet')

### Берем items только после мая

In [44]:
all_dfs = []
for i in tqdm(range(len(items_files)), desc='Processing files'):
    file_path = items_files[i]

    df = pl.read_parquet(
        file_path
    ).filter(pl.col('item_id').is_in(item_ids_from_may)) # from may
    all_dfs.append(df)

items_may_df = pl.concat(all_dfs)

Processing files:  86%|████████▌ | 86/100 [02:36<00:25,  1.81s/it]

: 

## Tracker

In [7]:
tr = pl.read_parquet(tracker_files[0])

In [9]:
tr['action_type'].unique()

action_type
str
"""to_cart"""
"""review_view"""
"""favorite"""
"""unfavorite"""
"""remove"""
"""view_description"""
"""page_view"""


In [ ]:
all_dfs = []
for i in tqdm(range(len(tracker_files)), desc='Processing files'):
    file_path = tracker_files[i]

    df = pl.read_parquet(
        file_path, 
        columns=['user_id', 'item_id', 'action_type', 'timestamp']
    ).filter(pl.col('created_timestamp') > pl.datetime(2025, 5, 21))
    all_dfs.append(df)

tracker = pl.concat(all_dfs)